In [5]:
%%capture
! pip install omegaconf timm mmcv imageio 
! pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [11]:
%%capture
!git clone https://github.com/rulixiang/afa.git

!rm -r afa

In [12]:
import os
os.chdir(f'./afa')
#os.getcwd()

In [13]:
import os
import sys
import random
import logging
import argparse
import datetime
import numpy as np
from tqdm import tqdm
from omegaconf import OmegaConf
import matplotlib.pyplot as plt

sys.path.append(".")

import torch
import torch.nn.functional as F
import torch.distributed as dist
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data.distributed import DistributedSampler


from wetr.PAR import PAR
from datasets import coco
from utils import evaluate, imutils
from wetr.model_attn_aff import WeTr
from utils.losses import get_aff_loss
from utils.optimizer import PolyWarmupAdamW
from utils.AverageMeter import AverageMeter
from utils.camutils import (cam_to_label, cams_to_affinity_label, ignore_img_box,
                            multi_scale_cam, multi_scale_cam_with_aff_mat,
                            propagte_aff_cam_with_bkg, refine_cams_with_bkg_v2,
                            refine_cams_with_cls_label)


In [14]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [15]:
parser = argparse.ArgumentParser()
parser.add_argument("--config",
                    default='configs/coco_attn_reg.yaml',
                    type=str,
                    help="config")
parser.add_argument("--pooling", default="gmp", type=str, help="pooling method")
parser.add_argument("--seg_detach", action="store_true", help="detach seg")
parser.add_argument("--work_dir", default=None, type=str, help="work_dir")
parser.add_argument("--local_rank", default=-1, type=int, help="local_rank")
parser.add_argument("--radius", default=8, type=int, help="radius")
parser.add_argument("--crop_size", default=224, type=int, help="crop_size")
parser.add_argument('--backend', default='nccl')

_StoreAction(option_strings=['--backend'], dest='backend', nargs=None, const=None, default='nccl', type=None, choices=None, help=None, metavar=None)

In [16]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [17]:
def get_down_size(ori_shape=(224,224), stride=16):
    h, w = ori_shape
    _h = h // stride + 1 - ((h % stride) == 0)
    _w = w // stride + 1 - ((w % stride) == 0)
    return _h, _w

In [ ]:
print(torch.cuda.set_device(1))

In [ ]:
dist.init_process_group(backend='nccl',)

In [24]:
wetr = WeTr(backbone='mit_b1',
            stride=[4, 2, 2, 1],
            num_classes=3,
            embedding_dim=256,
            pretrained=False,
            pooling='gmp',
           )

param_groups = wetr.get_param_groups()
par = PAR(num_iter=15, dilations=[1,2,4,8,12,24])

wetr.to(device)
par.to(device)

PAR()

In [27]:
wetr(torch.randn(1, 3, 224, 224).to(device))

RuntimeError: ignored